# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [12]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [37]:
IEX_CLOUD_API_TOKEN = ''

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol  = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [5]:
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': ')RLK( QLBLEGACT/GN NSEA DESSOAMAT', 'calculationPrice': 'close', 'open': 151.9, 'openTime': 1646409742441, 'openSource': 'acofiifl', 'close': 147.38, 'closeTime': 1708087386918, 'closeSource': 'oaffliic', 'high': 152.37, 'highTime': 1688039666060, 'highSource': 'ced l5e1 amiedrtnuipy e', 'low': 147.38, 'lowTime': 1707251948186, 'lowSource': 'iaecumd 1 eeli5prdeyn t', 'latestPrice': 150, 'latestSource': 'Close', 'latestTime': 'August 6, 2021', 'latestUpdate': 1707036238078, 'latestVolume': 56159719, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 151.6, 'delayedPriceTime': 1639984106401, 'oddLotDelayedPrice': 150.478, 'oddLotDelayedPriceTime': 1639190703833, 'extendedPrice': 151.03, 'extendedChange': -0.19, 'extendedChangePercent': -0.00127, 'extendedPriceTime': 1686261154705, 'previousClose': 150.51, 'previousVolume': 48238116, 'change': -0.7, 'changePercent': -0.00478, 'volume

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [ ]:
price  = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [4]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy


In [ ]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index = True
)

In [11]:

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(api_url)
    try:
        data = r.json()
    except ValueError:
        print("Response content is not valid JSON")
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

KeyboardInterrupt: 

In [8]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,162.30,47395430493,N/A
1,AAL,21.29,14131273939,N/A
2,AAP,207.14,13992971156,N/A
3,AAPL,151.92,2457167545393,N/A
4,ABBV,116.58,205899905610,N/A
...,...,...,...,...
500,YUM,137.37,41626982869,N/A
501,ZBH,154.00,31898020079,N/A
502,ZBRA,563.55,30227033708,N/A
503,ZION,57.34,9124861954,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [5]:
#making batch calls per hunderd usign yield
def chunks(stocks,n):
    for i in range(0, len(stocks), n):
        yield stocks[i:i+100]

In [6]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,161.810,49185308426,N/A
1,AAL,22.030,14159651350,N/A
2,AAP,212.410,13788423879,N/A
3,AAPL,146.300,2408927349153,N/A
4,ABBV,116.699,209387828846,N/A
...,...,...,...,...
500,YUM,134.903,39737253910,N/A
501,ZBH,148.240,31399997631,N/A
502,ZBRA,585.910,30772131556,N/A
503,ZION,57.620,9425320888,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [7]:
#accepting input from user
val = input('enter the portfolio amount: ')
try:
    portfolio_size = float(val)
except ValueError:
    print("That is not a number, try again")
    val = input('enter the portfolio amount: ')
    portfolio_size = float(val)

enter the portfolio amount: 1000000


In [8]:
#finding out how many shares to buy from each stock for a balanced s&p 500 portfolio
for i in range(0 , len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(portfolio_size / (final_dataframe.loc[i, 'Stock Price']))
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,161.810,49185308426,6180
1,AAL,22.030,14159651350,45392
2,AAP,212.410,13788423879,4707
3,AAPL,146.300,2408927349153,6835
4,ABBV,116.699,209387828846,8569
...,...,...,...,...
500,YUM,134.903,39737253910,7412
501,ZBH,148.240,31399997631,6745
502,ZBRA,585.910,30772131556,1706
503,ZION,57.620,9425320888,17355


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [32]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [33]:
final_dataframe.to_excel(writer ,'Recommended trades' , index = False)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    { 
        'num_format' : '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00', 
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [36]:
#using multiple lines === not a good method
'''
writer.sheets['Recommended trades'].set_columns('A':'A' ,18, string_format)
writer.sheets['Recommended trades'].set_columns('B':'B' ,18, dollar_format)
writer.sheets['Recommended trades'].set_columns('C':'C' ,18, dollar_format)
writer.sheets['Recommended trades'].set_columns('D':'D' ,18, dollar_format)
'''
#editing the title  of the column 
'''
writer.sheets['Recommended trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended trades'].write('C1','Market Capitalization',dollar_format)
writer.sheets['Recommended trades'].write('D1','Number of shares to buy',integer_format)

'''


#using loops for the same
#making the dictionary
#Ticker 	Stock Price 	Market Capitalization 	Number of shares to buy
column_format = {
    'A': ['Ticker' , string_format],
    'B': ['Stock Price' , dollar_format],
    'C': ['Market Capitalization' , dollar_format],
    'D': ['Number of shares to buy' , integer_format],
}


for column in column_format.keys():
    writer.sheets['Recommended trades'].set_column(f'{column}:{column}', 18, column_format[column][1])
    
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: